# Pos Tech - Fase 3 - Covid-19

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
from  pyspark.sql import Row, DataFrame
from  pyspark.sql.types import StringType, StructType, StructField, IntegerType
from  pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from  pyspark.sql import functions as F
from functools import reduce

import pyspark
import findspark

findspark.init()
from pyspark.sql import SparkSession

Spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
df = Spark.read.csv('/content/bq-results-20240930-215503-1727733352863.csv', sep=',', inferSchema= True,header= True)


In [ ]:
df.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- semana: integer (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- cor_ou_raca: integer (nullable = true)
 |-- escolaridade: integer (nullable = true)
 |-- febre: integer (nullable = true)
 |-- tosse: integer (nullable = true)
 |-- dor_de_garganta: integer (nullable = true)
 |-- dificuldade_para_respirar: integer (nullable = true)
 |-- dor_de_cabeca: integer (nullable = true)
 |-- dor_no_peito: integer (nullable = true)
 |-- nausa: integer (nullable = true)
 |-- nariz_entupido_ou_escorrendo: integer (nullable = true)
 |-- fadiga: integer (nullable = true)
 |-- dor_nos_olhos: integer (nullable = true)
 |-- perda_de_cheiro_ou_sabor: integer (nullable = true)
 |-- dor_muscular: integer (nullable = true)
 |-- atividade_profissao: integer (nullable = true)
 |-- semanas_trabalhadas: integer (nullable = true)
 |-- faixa_de_renda

In [ ]:
df = df.withColumn(
    "estado",
     when(col("estado") == "RO", "Rondônia")
    .when(col("estado") == "AC", "Acre")
    .when(col("estado") == "AM", "Amazonas")
    .when(col("estado") == "RR", "Roraima")
    .when(col("estado") == "PA", "Pará")
    .when(col("estado") == "AP", "Amapá")
    .when(col("estado") == "TO", "Tocantins")
    .when(col("estado") == "MA", "Maranhão")
    .when(col("estado") == "PI", "Piauí")
    .when(col("estado") == "CE", "Ceará")
    .when(col("estado") == "RN", "Rio Grande do Norte")
    .when(col("estado") == "PB", "Paraíba")
    .when(col("estado") == "PE", "Pernambuco")
    .when(col("estado") == "AL", "Alagoas")
    .when(col("estado") == "SE", "Sergipe")
    .when(col("estado") == "BA", "Bahia")
    .when(col("estado") == "MG", "Minas Gerais")
    .when(col("estado") == "ES", "Espírito Santo")
    .when(col("estado") == "RJ", "Rio de Janeiro")
    .when(col("estado") == "SP", "São Paulo")
    .when(col("estado") == "PR", "Paraná")
    .when(col("estado") == "SC", "Santa Catarina")
    .when(col("estado") == "RS", "Rio Grande do Sul")
    .when(col("estado") == "MS", "Mato Grosso do Sul")
    .when(col("estado") == "MT", "Mato Grosso")
    .when(col("estado") == "GO", "Goiás")
    .when(col("estado") == "DF", "Distrito Federal")
    .otherwise(col("estado"))  # Mantém a sigla original caso não haja correspondência
)


In [ ]:
df.show()

+----+---+------+------+-----+----+-----------+------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|sexo|cor_ou_raca|escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+----+-----------+------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------

In [ ]:
df = df.withColumn("sexo", col("sexo").cast("string"))

In [ ]:
df = df.withColumn(
    "sexo",
    when(col("sexo") == 1, "Homem").when(col("Sexo") == 2, "Mulher")
)
df.select("sexo").show(5)

+------+
|  sexo|
+------+
| Homem|
| Homem|
|Mulher|
| Homem|
|Mulher|
+------+
only showing top 5 rows



In [ ]:
df = df.withColumn("cor_ou_raca", col("cor_ou_raca").cast("string"))

In [ ]:
df = df.withColumn(
    "cor_ou_raca",
     when(col("cor_ou_raca") == 1, "Branca")
    .when(col("cor_ou_raca") == 2, "Preta")
    .when(col("cor_ou_raca") == 3, "Amarela")
    .when(col("cor_ou_raca") == 4, "Parda")
    .when(col("cor_ou_raca") == 5, "Indígena")
    .when(col("cor_ou_raca") == 9, "Ignorado")
)
df.select("cor_ou_raca").show(5)

+-----------+
|cor_ou_raca|
+-----------+
|      Parda|
|     Branca|
|     Branca|
|     Branca|
|      Parda|
+-----------+
only showing top 5 rows



In [ ]:
df = df.withColumn("escolaridade", col("escolaridade").cast("string"))

In [ ]:
df = df.withColumn(
    "escolaridade",
     when(col("Escolaridade") == 1, "Sem instrução")
    .when(col("Escolaridade") == 2, "Fundamental incompleto")
    .when(col("Escolaridade") == 3, "Fundamental completo")
    .when(col("Escolaridade") == 4, "Médio incompleto")
    .when(col("Escolaridade") == 5, "Médio completo")
    .when(col("Escolaridade") == 6, "Superior incompleto")
    .when(col("Escolaridade") == 7, "Superior completo")
    .when(col("Escolaridade") == 8, "Pós-graduação")
)
df.select("escolaridade").show(5)

+--------------------+
|        escolaridade|
+--------------------+
| Superior incompleto|
|      Médio completo|
|      Médio completo|
|      Médio completo|
|Fundamental incom...|
+--------------------+
only showing top 5 rows



In [ ]:
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn("febre", col("febre").cast("string"))

In [ ]:
df = df.withColumn(
    "febre",
    when(col("febre") == 1, "Sim")
    .when(col("febre") == 2, "Não")
    .when(col("febre") == 3, "Não sabe")
    .otherwise("Desconhecido")  # Opcional, para valores fora do esperado
)

In [ ]:
df = df.withColumn("tosse", col("tosse").cast("string"))

In [ ]:
df = df.withColumn(
    "tosse",
     when(col("tosse") == 1, "Sim")
    .when(col("tosse") == 2, "Não")
    .when(col("tosse") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dor_de_garganta",
     when(col("dor_de_garganta") == 1, "Sim")
    .when(col("dor_de_garganta") == 2, "Não")
    .when(col("dor_de_garganta") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dificuldade_para_respirar",
     when(col("dificuldade_para_respirar") == 1, "Sim")
    .when(col("dificuldade_para_respirar") == 2, "Não")
    .when(col("dificuldade_para_respirar") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dor_de_cabeca",
     when(col("dor_de_cabeca") == 1, "Sim")
    .when(col("dor_de_cabeca") == 2, "Não")
    .when(col("dor_de_cabeca") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dor_no_peito",
     when(col("dor_no_peito") == 1, "Sim")
    .when(col("dor_no_peito") == 2, "Não")
    .when(col("dor_no_peito") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "nausa",
     when(col("nausa") == 1, "Sim")
    .when(col("nausa") == 2, "Não")
    .when(col("nausa") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "nariz_entupido_ou_escorrendo",
     when(col("nariz_entupido_ou_escorrendo") == 1, "Sim")
    .when(col("nariz_entupido_ou_escorrendo") == 2, "Não")
    .when(col("nariz_entupido_ou_escorrendo") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "fadiga",
     when(col("fadiga") == 1, "Sim")
    .when(col("fadiga") == 2, "Não")
    .when(col("fadiga") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dor_nos_olhos",
     when(col("dor_nos_olhos") == 1, "Sim")
    .when(col("dor_nos_olhos") == 2, "Não")
    .when(col("dor_nos_olhos") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "perda_de_cheiro_ou_sabor",
     when(col("perda_de_cheiro_ou_sabor") == 1, "Sim")
    .when(col("perda_de_cheiro_ou_sabor") == 2, "Não")
    .when(col("perda_de_cheiro_ou_sabor") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn(
    "dor_muscular",
     when(col("dor_muscular") == 1, "Sim")
    .when(col("dor_muscular") == 2, "Não")
    .when(col("dor_muscular") == 3, "Não sabe")
    .otherwise("Desconhecido")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+--------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+---------------------

In [ ]:
df = df.withColumn("atividade_profissao", col("atividade_profissao").cast("string"))

In [ ]:
df = df.withColumn(
    "atividade_profissao",
    when(col("atividade_profissao") == 1, "Agricultura")
    .when(col("atividade_profissao") == 2, "Mineração")
    .when(col("atividade_profissao") == 3, "Indústria")
    .when(col("atividade_profissao") == 4, "Energia")
    .when(col("atividade_profissao") == 5, "Construção")
    .when(col("atividade_profissao") == 6, "Comércio")
    .when(col("atividade_profissao") == 7, "Reparo")
    .when(col("atividade_profissao") == 8, "Transporte")
    .when(col("atividade_profissao") == 9, "Logística")
    .when(col("atividade_profissao") == 10, "Logística")
    .when(col("atividade_profissao") == 11, "Turismo")
    .when(col("atividade_profissao") == 12, "Alimentação")
    .when(col("atividade_profissao") == 13, "Comunicação")
    .when(col("atividade_profissao") == 14, "Financeiro")
    .when(col("atividade_profissao") == 15, "Imobiliário")
    .when(col("atividade_profissao") == 16, "Consultoria")
    .when(col("atividade_profissao") == 17, "Serviços")
    .when(col("atividade_profissao") == 18, "Administração")
    .when(col("atividade_profissao") == 19, "Educação")
    .when(col("atividade_profissao") == 20, "Saúde")
    .when(col("atividade_profissao") == 21, "Organizações")
    .when(col("atividade_profissao") == 22, "Cultura")
    .when(col("atividade_profissao") == 23, "Estética")
    .when(col("atividade_profissao") == 24, "Doméstico")
    .when(col("atividade_profissao") == 25, "Diversos")
    .otherwise("Desconhecido"))

In [ ]:
df = df.withColumn(
    "faixa_de_renda",
    when(col("faixa_de_renda") == 0, "0 - 100")
    .when(col("faixa_de_renda") == 1, "101 - 300")
    .when(col("faixa_de_renda") == 2, "301 - 600")
    .when(col("faixa_de_renda") == 3, "601 - 800")
    .when(col("faixa_de_renda") == 4, "801 - 1.600")
    .when(col("faixa_de_renda") == 5, "1.601 - 3.000")
    .when(col("faixa_de_renda") == 6, "3.001 - 10.000")
    .when(col("faixa_de_renda") == 7, "10.001 - 50.000")
    .when(col("faixa_de_renda") == 8, "50.001 - 100.000")
    .when(col("faixa_de_renda") == 9, "Mais de 100.000")
    .otherwise("Não especificado")

    )
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+------------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+-----------------

In [ ]:
df = df.withColumn(
    "Faixa_Etaria",
    when(col("idade") < 18, "Menor de 18").when((col("idade") >= 18) & (col("idade") < 40), "18-39")
    .when((col("idade") >= 40) & (col("idade") < 60), "40-59").when(col("idade") >= 60, "60+")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+------------------+------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|Faixa_Etaria|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-----

In [ ]:
df = df.withColumn(
    "teste_realizado",
     when(col("teste_realizado") == 1, "Sim")
    .when(col("teste_realizado") == 2, "Não")
    .when(col("teste_realizado") == 9, "Ignorado")
    .otherwise("Não Aplicável")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+------------------+------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|Faixa_Etaria|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-----

In [ ]:
df = df.withColumn(
    "resultado_teste",
     when(col("resultado_teste") == 1, "Positivo")
    .when(col("resultado_teste") == 2, "Negativo")
    .when(col("resultado_teste") == 2, "Inconclusivo")
    .when(col("resultado_teste") == 2, "Ainda não recebeu o resultado")
    .when(col("resultado_teste") == 9, "Ignorado")
    .otherwise("Não Aplicável")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+------------------+------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|Faixa_Etaria|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-----

In [ ]:
df = df.withColumn(
    "temporariamente_afastado_trabalho",
     when(col("temporariamente_afastado_trabalho") == 1, "Sim")
    .when(col("temporariamente_afastado_trabalho") == 2, "Não")
    .otherwise("Não Aplicável")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+------------------+------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|motivo_afastamento|Faixa_Etaria|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-----

In [ ]:
df = df.withColumn(
    "motivo_afastamento",
     when(col("motivo_afastamento") == 1, "Quarentena, isolamento, distanciamento social ou férias coletivas")
    .when(col("motivo_afastamento") == 2, "Outro motivo")
    .when(col("motivo_afastamento") == 3, "Licença")
    .when(col("motivo_afastamento") == 4, "Licença")
    .when(col("motivo_afastamento") == 5, "Licença")
    .when(col("motivo_afastamento") == 6, "Outro motivo")
    .when(col("motivo_afastamento") == 7, "Fatores ocasionais (mau tempo, paralisação nos serviços de transportes, etc.)")
    .when(col("motivo_afastamento") == 8, "Outro motivo")
    .otherwise("Não Aplicável")
)
df.show()

+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-------------+------------------------+------------+-------------------+-------------------+----------------+---------------+---------------+---------------------------------+--------------------+------------+
| ano|mes|semana|estado|idade|  sexo|cor_ou_raca|        escolaridade|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausa|nariz_entupido_ou_escorrendo|fadiga|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|atividade_profissao|semanas_trabalhadas|  faixa_de_renda|teste_realizado|resultado_teste|temporariamente_afastado_trabalho|  motivo_afastamento|Faixa_Etaria|
+----+---+------+------+-----+------+-----------+--------------------+-----+-----+---------------+-------------------------+-------------+------------+-----+----------------------------+------+-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
caminho = '/content/drive/My Drive/Pós FIAP - Fase 3/db_bq_spark_covid_19'

df.write.csv(caminho, header=True)